In [1]:
from time import time as now

import numpy as np
import pandas as pd
import utils

COBRA_PATH = 'data/iJO1366.json'
BIOMASS_ID = 'Ec_biomass_iJO1366_WT_53p95M'


--------------------------------------------
--------------------------------------------

Using license file C:\Users\riheme\gurobi.lic
Academic license - for non-commercial use only


In [2]:
# generates a stacked DataFrame that shares values in columns
def gen_rows( data, **kwargs ):
    # we receive fluxes, log concentrations, or free energies
    # correctly identify that type of value
    labels = { 'value_type': data.columns[0], **kwargs }
    # the original data has to be renamed, so when these rows are stacked they don't create additional columns
    mapper = { data.index.name: 'id', data.columns[0]: 'value', data.columns[1]: 'tag' }
    # create new DataFrame and rename it
    rows = pd.DataFrame( data ).reset_index()
    rows.rename( columns = mapper, inplace = True )
    # add labels as columns
    for i, (col, value) in enumerate(labels.items()):
        rows.insert( i, col, value )
    return rows

# TFA

1. Load necessary data (thermo DB, lexicon, compartment data, LC bounds, rate bounds)

In [3]:
thermo_data, lexicon, compartment_data = utils.load_data(
    'data/thermo_data.thermodb',
    'data/lexicon.csv',
    'data/compartment_data.json' )
bounds = pd.read_csv('data/bounds.csv')
bounds.head()

,bound_type,strain,ko,media,time_point,id,lb,ub
0,log_concentration,0,GLCptspp,Glc,0,23dpg,-7.860059,-7.424463
1,log_concentration,0,GLCptspp,Glc,0,6pgc,-8.785181,-8.248013
2,log_concentration,0,GLCptspp,Glc,0,Lcystin,-9.206868,-8.559319
3,log_concentration,0,GLCptspp,Glc,0,Pool_2pg_3pg,-6.367578,-5.712819
4,log_concentration,0,GLCptspp,Glc,0,accoa,-8.114400,-7.619540


2. Run simulations!

In [4]:
start = now()

results = []

media = 'Glc'
media_mask = bounds['media'] == media
for ko_id in bounds[ media_mask ]['ko'].unique():
    ko_mask = bounds['ko'] == ko_id
    for strain in bounds[ ko_mask & media_mask ]['strain'].unique():
        strain_mask = bounds['strain'] == strain
        for time in bounds[ ko_mask & media_mask & strain_mask ]['time_point'].unique():
            mask = ko_mask & media_mask & strain_mask & ( bounds['time_point'] == time )
            name = f'Ecoli {ko_id} ({media}) {strain} @ t = {time}'
            print( '\n'.join( ['#'*80, name, f'({(now()-start)/60:.2f} min)', '#'*80] ) )

            # get log concentration and reaction bounds
            lc_bounds = bounds[ mask & ( bounds['bound_type'] == 'log_concentration' ) ]
            rxn_bounds = bounds[ mask & ( bounds['bound_type'] == 'reaction' ) ]

            # create, adjust, and run thermodynamic model
            tmodel = utils.create_model( name, COBRA_PATH, thermo_data, lexicon, compartment_data, BIOMASS_ID )
            utils.adjust_model( tmodel, lc_bounds, rxn_bounds )
            tmodel, relax_table = utils.optimize( tmodel, relax = 'dgo' )

            # store free energies, log concentrations, and fluxes
            cols = dict( strain = strain, ko = ko_id, media = media, time_point = time )
            results.append( gen_rows( utils.get_flux( tmodel ), **cols ) )
            results.append( gen_rows( utils.get_delta_g( tmodel ), **cols ) )
            results.append( gen_rows( utils.get_log_concentration( tmodel ), **cols ) )
            results.append( gen_rows( relax_table, **cols ) )

results = pd.concat(results, ignore_index = True)
results.to_csv('data/results.csv', index = False)

print(f'Elapsed time: {(now()-start)/3600:.2f} h')

################################################################################
Ecoli GLCptspp (Glc) 0 @ t = 0
(0.00 min)
################################################################################
Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmpq9zgt7kn.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20334 nonzeros
Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmpiwm6dlmu.lp
Reading time = 0.03 seconds
: 1807 rows, 5170 columns, 20334 nonzeros


2020-11-29 12:05:00,407 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2020-11-29 12:05:00,409 - thermomodel_None - INFO - # Model preparation starting...
2020-11-29 12:05:02,627 - thermomodel_None - WARNING - Warning : NULL/U
2020-11-29 12:05:02,627 - thermomodel_None - WARNING - Warning : NULL/L
2020-11-29 12:05:02,627 - thermomodel_None - WARNING - Warning : NULL/L
2020-11-29 12:05:02,627 - thermomodel_None - WARNING - Warning : NULL/U
2020-11-29 12:05:02,627 - thermomodel_None - WARNING - Warning : NULL/L
2020-11-29 12:05:02,627 - thermomodel_None - WARNING - Warning : NULL/L
2020-11-29 12:05:02,845 - thermomodel_None - WARNING - Warning : NULL/U
2020-11-29 12:05:02,845 - thermomodel_None - WARNING - Warning : NULL/L
2020-11-29 12:05:02,845 - thermomodel_None - WARNING - Warning : NULL/L
2020-11-29 12:05:02,845 - thermomodel_None - WARNING - Warning : NULL/U
2020-11-29 12:05:02,845 - thermomodel_None - WARNING - Warning : NULL/L
2020-11

Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmp6zabmc9k.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros
Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmpez_9xxz_.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2020-11-29 12:05:51,103 - thermomodel_Ecoli GLCptspp (Glc) 0 @ t = 0 - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
C:\Users\riheme\Anaconda3\lib\site-packages\sympy\__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.numbers with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.numbers instead. See
https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmp4kzgf63s.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros
Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmpw8neqw0d.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


...

Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmpzczau856.lp
Reading time = 0.05 seconds
: 1807 rows, 5170 columns, 20304 nonzeros
Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmprdn6qkk1.lp
Reading time = 0.07 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2020-11-29 16:05:34,489 - thermomodel_Ecoli wt (Glc) 2 @ t = 11 - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmp7tbytn71.lp
Reading time = 0.06 seconds
: 1807 rows, 5170 columns, 20304 nonzeros
Read LP format model from file C:\Users\riheme\AppData\Local\Temp\tmpdoa5_3kt.lp
Reading time = 0.06 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2020-11-29 16:06:45,955 - thermomodel_Ecoli wt (Glc) 2 @ t = 11 - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2020-11-29 16:07:12,179 - thermomodel_Ecoli wt (Glc) 2 @ t = 11 - INFO - Adding slack constraints
adding slacks: 100%|███████████████████████████████████████████████████████████████| 1824/1824 [01:34<00:00, 19.21it/s]
2020-11-29 16:08:51,669 - thermomodel_Ecoli wt (Glc) 2 @ t = 11 - INFO - Optimizing slack model
2020-11-29 16:09:03,193 - thermomodel_Ecoli wt (Glc) 2 @ t = 11 - INFO - Extracting relaxation
applying slack: 100%|███████████████████████████████████████████████████████████| 2585/2585 [00:00<00:00, 17242.12it/s]
2020-11-29 16:09:03,558 - thermomodel_Ecoli wt (Glc) 2 @ t = 11 - INFO - Testing relaxation


Elapsed time: 4.07 h


3. Inspect results

In [5]:
results.head()

,value_type,strain,ko,media,time_point,id,value,tag
0,flux,0,GLCptspp,Glc,0,DM_4CRSOL,4.467703e-05,Exchange
1,flux,0,GLCptspp,Glc,0,DM_5DRIB,4.627979e-05,Exchange
2,flux,0,GLCptspp,Glc,0,DM_AACALD,0.000000e+00,Exchange
3,flux,0,GLCptspp,Glc,0,DM_AMOB,4.006909e-07,Exchange
4,flux,0,GLCptspp,Glc,0,DM_MTHTHF,2.684629e-04,Exchange
